# GlusterFS安装报告（基于Kubernetes）
## 什么是GlusterFS
>**GlusterFS (Gluster File System)** 是一个开源的分布式文件系统，主要由 Z RESEARCH 公司负责开发。GlusterFS 是 Scale-Out 存储解决方案 Gluster 的核心，具有强大的横向扩展能力，通过扩展能够支持数PB存储容量和处理数千客户端。GlusterFS 借助 TCP/IP 或 InfiniBand RDMA 网络将物理分布的存储资源聚集在一起，使用单一全局命名空间来管理数据。GlusterFS 基于可堆叠的用户空间设计，可为各种不同的数据负载提供优异的性能。

<p>GlusterFS由存储服务器（Brick Server），客户端以及NFS/Samba存储网关组成。GlusterFS最大的设计特点就是其没有元数据（metadata）存储组件。这样的设计对整个系统性能，可靠性和稳定性的提升都具有非常重大的意义。其总体架构如下图所示：</p>
<img style="width:75%" src="https://github.com/yhswjtuILMARE/kubernetes-DB/blob/master/images/structure_gfs.png" />
<p>接下来简要说明一下Glusterfs中的一些要用到的专业术语：<ul><li><b>Brick</b>：最基本的存储单元，供客户端挂载使用。</li></ul><ul><li><b>Fuse</b>：类Unix操作系统中的一个可动态加载的模块，用以允许用户不用修改内核即可创建自己的文件系统。</li></ul><ul><li><b>PV(phyical volume)</b>：即物理卷，是<b>lvm（逻辑卷管理系统）</b>中的概念，类似于物理磁盘上的分区，其基本组成单位是物理快（PE），其默认大小为4MB。</li><li><b>VG(volumes group)</b>：即<b>卷组</b>，由物理卷构成，是逻辑卷的组成部分。</li></ul><ul><li><b>LV(Logistic Volume)逻辑卷</b>：是lvm中最上层的文件卷，由若干个卷组中的物理卷构成，其大小可以动态指定，也能够扩大或缩小。</li></ul>Glusterfs一共支持七种挂载卷的形式，包括分布式卷，条带卷，副本卷等多种类型，具体的内容可参见<a href="https://www.ibm.com/developerworks/cn/opensource/os-cn-glusterfs-docker-volume/index.html?lnk=hm" target="_blank">基于 GlusterFS 实现 Docker 集群的分布式存储</a>这篇文章。</p>

## 基于Kubernetes的GlusterFS的安装
### 安装环境
<p><pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
NAME   STATUS    ROLES    VERSION   EXTERNAL-IP    OS-IMAGE             KERNEL-VERSION     CONTAINER-RUNTIME
lab1   Ready     node     v1.8.2    172.19.0.131   Ubuntu 16.04.4 LTS   4.4.0-62-generic   docker://17.11.0-ce
lab2   Ready     node     v1.8.2    172.19.0.132   Ubuntu 16.04.2 LTS   4.4.0-62-generic   docker://17.11.0-ce
lab3   Ready     node     v1.8.2    172.19.0.133   Ubuntu 16.04.2 LTS   4.4.0-62-generic   docker://17.11.0-ce
lab4   Ready     master   v1.8.2    172.19.0.134   Ubuntu 16.04.4 LTS   4.4.0-62-generic   docker://17.11.0-ce
</pre>可以看到，安装环境中一共有四个节点，其中lab4为主节点，其余三个节点为从节点。每个节点上都运行Ubuntu16.04的操作系统。<b>（注意：GlusterFS安装要求必须至少有三个存储节点）</b></p>

### GlusterFS集群搭建
<p>如果计算机中没有预先编写好的yaml文件，可以从git上下载官方的配置仓库，在cmd中运行<code style="font-family:Consolas;padding:5px 5px 5px 5px;background-color:#F4F4F4">git clone https://github.com/gluster/gluster-kubernetes</code>即可下载到本地，其具体的yaml文件在配置仓库的<code style="font-family:Consolas;padding:5px 5px 5px 5px;background-color:#F4F4F4">./deploy/kube-templates</code>中。虽然GlusterFS提供安装指南，但还是推荐使用预先编写好的yaml文件配置GlusterFS，因为官方仓库里下载的配置文件还需要一定的更改才能够正确运行在本地集群上。下面的安装全都假定操作者已经拥有预先编写好的yaml文件或者已经对git上下载的配置文件进行了正确修改。</p>
<p>首先查看所有四个节点的hosts文件，需要在各个节点的hosts文件中添加其余节点的主机名，ip地址映射列表。例如在笔者所在的集群环境下hosts文件的内容：
<pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
root@lab4:/# cat /etc/hosts
127.0.0.1	localhost
172.19.0.134    lab4
172.19.0.131    lab1
172.19.0.132    lab2
172.19.0.133    lab3
</pre>配置完成hosts文件后需要在各个运行GlusterFS的节点上下载Glusterfs-client客户端进程。<b>（注意：不是glusterfs-server进程）</b>下载Glusterfs-client之前首先要更新软件源，在这里笔者更新的软件源为3.12版本，然后更新软件源。代码清单如下：
<pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
root@lab3:/# echo deb http://ppa.launchpad.net/gluster/glusterfs-3.12/ubuntu xenial main >> /etc/apt/sources.list
root@lab3:/# echo deb-src http://ppa.launchpad.net/gluster/glusterfs-3.12/ubuntu xenial main >> /etc/apt/sources.list
root@lab3:/# apt-get update
</pre><b>（注意：软件源更新时可能会出现403错误，这并不影响下载glusterfs客户端程序，不用理会）</b>更新完成之后就可以进行客户端程序的下载了，代码清单如下：
<pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
root@lab3:/# apt-get install glusterfs-client
</pre>通常下载完成之后glusterfs-client程序就自动开始运行，不需要手动启动。需要特别说明的是只有需要运行作为GlusterFS存储节点的物理机需要下载并安装GlusterFS-client，例如在本例中lab4作为kubernetes的主节点并不需要承担存储任务，因此就没有安装glusterfs-client程序（当然，装上也并不会有错）。</p>
<p>执行到这一步，存储节点上的glusterfs设置就已经完成了。接下来的操作都主要在kubernetes主节点（以下均称为主节点，该节点不承担glusterfs存储任务），在本例中即lab4节点上进行。首先查看主节点上glusterfs集群的全部yaml文件：<ul><li><b>glusterfs-daemonset.yaml：</b>该文件用于创建glusterfs管理程序容器所在pod，可以理解成glusterfs的服务端，每一个glusterfs存储节点上必须有一个该类型的pod在运行。需要特别说明的是，该daemonset中采用的镜像是基于centos的，不过这并不妨碍其运行在Ubuntu系统上。</li></ul><ul><li><b>heketi-deployment.yaml：</b>该文件用于创建heketi容器，该容器内运行kehekti服务进程，该进程为用户提供RESTful API，允许用户操作glusterfs集群，这便于我们自动化地管理GlusterFS集群。</li></ul><ul><li><b>heketi-service-account.yaml：</b>该文件用于创建heketi服务容器所需的ServiceAccount，以及该ServiceAccount所需的用户角色，权限。由于heketi提供RESTful API允许用户管理GlusterFS集群，这就必然要求heketi操作kubernetes的某些api。基于kubernetes本身的授权机制，需要为heketi创建服务账户，并且绑定一定的权限。</li></ul><ul><li><b>topology.json：</b>该文件用于描述GlusterFS集群的存储节点的拓扑结构以及所用的物理卷位置，该文件需要根据集群的实际情况编写。</li></ul><ul><li><b>glusterfs-storageClass.yaml：</b>该文件用于创建storageclass对象，该对象用于创建kubernetes中的pv对象。</li></ul><ul><li><b>glusterfs-pvc.yaml：</b>该文件用于创建pvc对象，在创建过程中会自动创建pv对象。</li></ul>在开始GlusterFS安装前还需要做一项准备工作，由上面的文件清单可以看出glusterfs管理进程的pod是由daemonset对象建立的，之所以由该对象建立而不是ReplicationController对象建立是因为daemonset对象可以确保在每个目标物理机上只分配一个pod，该机制是基于node标签的，在本例中需要为每个运行glusterfs管理进程pod的节点都打上<code style="font-family:Consolas;padding:5px 5px 5px 5px;background-color:#F4F4F4">storagenode=glusterfs</code>的标签，代码清单如下：
<pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
root@lab1:/# kubectl label node lab1 storagenode=glusterfs
root@lab2:/# kubectl label node lab2 storagenode=glusterfs
root@lab3:/# kubectl label node lab3 storagenode=glusterfs
</pre>标签设置完成后就可以正式开始GlusterFS安装了，首先在各个存储节点上运行GlusterFS管理进程pod，代码清单如下：
<pre style="font-family:Consolas;background-color:#F6F8FA;padding:20px 15px 20px 15px">
root@lab4:/home/yanghang/source/database/glusterFS# kubectl create -f glusterfs-daemonset.yaml
root@lab4:/home/yanghang/source/database/glusterFS# daemonset "glusterfs" created
root@lab4:/home/yanghang/source/database/glusterFS# kubectl get po -o wide
NAME                            READY     STATUS    RESTARTS   AGE       IP               NODE
glusterfs-7fpsq                 1/1       Running   0          1d        172.19.0.132     lab2
glusterfs-fg6z9                 1/1       Running   0          1d        172.19.0.133     lab3
glusterfs-w68w4                 1/1       Running   0          1d        172.19.0.131     lab1
</pre>可以看到，三个GlusterFS管理程序容器的pod都已经正常启动，若READY容器的数目没有达到1则可以多等待一会儿。</p>
<p>接下来开始启动heketi服务。首先设置heketi服务的ServiceAccount以及设置相关权限，代码清单如下：